In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import time
from scipy.optimize import fsolve, minimize
from scipy.integrate import quad
from scipy import linspace, meshgrid, arange, empty, concatenate, newaxis, shape
from collections import deque
import nbimporter

## MDP functions

In [2]:
# Following functions solve Bellman optimality equation
def A(model_pars, state, V):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    # Calculating A
    A_val = (n1 + n2) + \
            (lam*alpha*(V[n1+1, n2] - V[n1, n2]) if n1 < M else 0) + \
            (lam*(1-alpha)*(V[n1, n2+1] - V[n1, n2]) if n2 < M else 0)
    return A_val
    
    
def H_and_action(model_pars, state, V):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    
    # Evaluating best continuous action
    # min_cost = minimize(cost, 0.5 , method="L-BFGS-B", args = (model_pars, state, V), bounds = [(0,1)])
    # a_optimal = min_cost.x
    # H_optimal = V[n1, n2] + float(min_cost.fun)
    
    # Evaluating best discrete action from [0, 1/cores, 2/cores, ..., 1]
    costs_given_action = np.array([cost(i/cores, model_pars, state, V) for i in  range(cores+1)])
    index = np.argmin(costs_given_action)
    a_optimal = index/cores
    H_optimal = V[n1, n2] + costs_given_action[index]
    return H_optimal, a_optimal
    
    
def cost(action, model_pars, state, V):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    # Calculating cost
    cost = (min(n1, cores*action)*mu*speed_up(p1, cores*action/n1)*(V[n1-1, n2] - V[n1, n2]) if n1 > 0 else 0) + \
           (min(n2, cores*(1-action))*mu*speed_up(p2, cores*(1-action)/n2)*(V[n1, n2-1] - V[n1, n2]) if n2 > 0 else 0)
    return cost


def solve_MDP(model_pars):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = model_pars
    # We define old and new value function and update them.
    old_V = np.full((M+1, M+1), 1.0, dtype=np.float64)
    new_V = np.full((M+1, M+1), 1.0, dtype=np.float64)
    # Change in old_V and new_V should eventually converge to average cost per unit time.
    change_V = np.zeros((M+1, M+1))
    # Stores optimal action
    action = np.zeros((M+1, M+1))
    
    epsilon = 0.01 # Maximum permissible error
    delta = math.inf
    
    iteration = 1
    
    while delta >= epsilon:
        fraction_change = 1
        for i in range(M+1):
            for j in range(M+1):
                term_1 = A(model_pars, [i,j], old_V)
                term_2, a = H_and_action(model_pars, [i,j], old_V)
                action[i,j] = a
                new_V[i,j] = term_1 + float(term_2)
                fraction_change = max(fraction_change, (new_V[i,j] - old_V[i,j])/old_V[i,j] if old_V[i,j] > 0 else 1)
        change_V = new_V.copy() - old_V.copy()
        delta = change_V.max() - change_V.min()
        # print("Iteration = ", iteration, " Delta = ", delta)
        old_V = new_V.copy()
        iteration += 1
        
    return new_V, action


def bellman_optimal_policy(pars):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = pars
    # Scales the parameters so that sum of arrival and departure rates is smaller than 1.
    scale = lam + M*mu*(speed_up(p1, cores) + speed_up(p2, cores))
    # scale = lam + mu*np.max([min(M, c)*speed_up(p1, c) + min(M, cores-c)*speed_up(p2, cores-c) for c in range(cores+1)])
    model_pars = [lam/scale, mu/scale, cores, p1, p2, alpha, M]
    # Optimal value function and optimal policy
    V_optimal, pi_optimal  = solve_MDP(model_pars)
    relative_V_optimal = V_optimal - V_optimal[0,0]
    # Calculate relative Q values
    relative_Q_optimal = np.zeros((M+1, M+1, cores+1))
    for i in range(M+1):
        for j in range(M+1):
            for k in range(cores+1):
                probs = transition_probabilities(model_pars, [i,j], k/cores)
                future_cost = 0
                future_cost += probs[0]*relative_V_optimal[i+1, j] if i < M else 0
                future_cost += probs[1]*relative_V_optimal[i, j+1] if j < M else 0
                future_cost += probs[2]*relative_V_optimal[i-1, j] if i > 0 else 0
                future_cost += probs[3]*relative_V_optimal[i, j-1] if j > 0 else 0
                future_cost += probs[4]*relative_V_optimal[i, j]
                relative_Q_optimal[i, j, k] = (i+j) + future_cost
    
    return pi_optimal, relative_V_optimal, relative_Q_optimal

## General functions

In [3]:
def speed_up(p, n_cores):
    rate = 1/(1 - p*(1 - 1/max(1, n_cores)))
    return rate


# Returns transition probabilities from any state (n1, n2)
def transition_probabilities(model_pars, state, action):
    lam, mu, cores, p1, p2, alpha, M = model_pars
    [n1, n2] = state
    
    prob_1 = lam*alpha if n1 < M else 0
    prob_2 = lam*(1-alpha) if n2 < M else 0
    prob_3 = min(n1, cores*action)*mu*speed_up(p1, cores*action/n1) if n1 > 0 else 0
    prob_4 = min(n2, cores*(1-action))*mu*speed_up(p2, cores*(1-action)/n2) if n2 > 0 else 0
    prob_5 = 1 - prob_1 - prob_2 - prob_3 - prob_4
    return [prob_1, prob_2, prob_3, prob_4, prob_5]


# Given current state, it samples the next state under the optimal core allocation policy
# which is evaluated by solving the Bellman optimality equation
def sample_next_state(model_pars, current_state, action):
    [n1, n2] = current_state
    
    possible_next_states = [[n1+1, n2], [n1, n2+1], [n1-1, n2], [n1, n2-1], [n1, n2]]
    indices = [0, 1, 2, 3, 4]
    probabilities = transition_probabilities(model_pars, current_state, action)
    next_state = np.array(possible_next_states[np.random.choice(indices, size = 1, p = probabilities)[0]])
    return next_state


# Heatmap of two policies, and their difference.
def visualize_policies(policy_1, policy_2):
    policy_diff = policy_1 - policy_2
    
    # Policy 1 heatmap
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    sns.heatmap(policy_1, ax=axes[0], cmap="coolwarm", annot=False, cbar=True)
    axes[0].set_title("Heatmap of Policy 1")
    axes[0].invert_yaxis()  # This makes (0,0) appear at bottom left
    # Policy 2 heatmap
    sns.heatmap(policy_2, ax=axes[1], cmap="coolwarm", annot=False, cbar=True)
    axes[1].set_title("Heatmap of Policy 2")
    axes[1].invert_yaxis()  # This makes (0,0) appear at bottom left
    # Policy difference heatmap
    sns.heatmap(policy_diff, ax=axes[2], cmap="coolwarm", annot=False, cbar=True)
    axes[2].set_title("Policy difference")
    axes[2].invert_yaxis()  # This makes (0,0) appear at bottom left
    plt.show()

In [4]:
# How Q-values vary across the state space given action
def plot_q_heatmap(Q, cores, action):
    fig, ax = plt.subplots(figsize=(18, 15))
    data = Q[:, :, int(cores * action)]
    sns.heatmap(data, cmap="viridis", annot=False, ax=ax)
    # Move x-axis to top
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    # Grid coordinate labels
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            ax.text(j + 0.5, i + 0.5, f"{np.round(Q[i,j], 1)}", ha='center', va='center', fontsize=5, color='white')
    ax.set_title(f"Q-values Heatmap for action = {action}", pad=30)
    ax.set_xlabel("$n_2$ (Good Jobs)")
    ax.set_ylabel("$n_1$ (Bad Jobs)")
    plt.show()

    
# How Q-values vary with state for a fixed action
def plot_q_surface(Q, cores, action):
    n1_vals, n2_vals = np.meshgrid(range(Q.shape[0]), range(Q.shape[1]), indexing="ij")
    q_vals = Q[:, :, int(cores*action)]

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(n1_vals, n2_vals, q_vals, cmap="viridis")

    ax.set_title(f"Q-values Surface Plot for action = {action}")
    ax.set_xlabel("$n_1$ (Bad Jobs)")
    ax.set_ylabel("$n_2$ (Good Jobs)")
    ax.set_zlabel("Q-value")
    plt.show()

    
# How Q-values change across actions for a fixed state (i,j)
def plot_q_vs_actions(Q, state):
    [n1, n2] = state
    actions = np.arange(Q.shape[2])
    q_values = Q[n1, n2, :]

    plt.figure(figsize=(8, 5))
    plt.plot(actions, q_values, marker='o', linestyle='-', color='b')
    plt.title(f"Q-values for State ({n1}, {n2})")
    plt.xlabel("Action k")
    plt.ylabel("Q-value")
    plt.grid(True)
    plt.show()

## Q-Learning functions

In [5]:
def Q_learning(model_pars, initial_state, n_iters):
    lam, mu, cores, p1, p2, alpha, M = model_pars
    current_state = initial_state
    next_state = np.zeros(2)
    
    Q_values = np.zeros((M+1, M+1, cores+1))
    count_state_action_visits = np.zeros((M+1, M+1, cores+1))
    long_run_average_cost = 0
    state_history = [current_state]
    long_run_average_cost_history = [long_run_average_cost]
    
    for t in range(n_iters):
        # Given current state, we take action
        epsilon = 1 / math.pow((1+t), 0.5)
        ber = np.random.binomial(1, epsilon)
        action = np.argmin(np.array(Q_values[current_state[0], current_state[1], :]))/cores if ber == 0 else np.random.randint(low = 0, high = cores+1, size = 1)[0]/cores
        # Count state-action frequency
        count_state_action_visits[current_state[0], current_state[1], int(cores*action)] += 1
        # After taking action, we observe immediate cost
        immediate_cost = current_state[0] + current_state[1]
        # Then, we observe future state
        next_state = sample_next_state(model_pars, current_state, action)
        state_history.append(next_state)
        # Long run average cost update
        beta = 1/math.pow((1+t),1)
        long_run_average_cost = (1-beta)*long_run_average_cost + beta*immediate_cost
        long_run_average_cost_history.append(long_run_average_cost)
        # Q update
        alpha = 1/(1 + 0.01*t)
        Q_values[current_state[0], current_state[1], int(cores*action)] \
                = (1-alpha)*Q_values[current_state[0], current_state[1], int(cores*action)] \
                   + alpha*(immediate_cost - long_run_average_cost + np.min(np.array(Q_values[next_state[0], next_state[1], :])) - Q_values[0, 0, 0])
        # State update
        current_state = next_state
    # Evaluating optimal policy as argmin of Q-values    
    optimal_actions = np.zeros((M+1, M+1))
    for n1 in range(M+1):
        for n2 in range(M+1):
            argmin_index = np.argmin(np.array(Q_values[n1, n2, :]))
            optimal_actions[n1, n2] = argmin_index/cores
            
    return Q_values, optimal_actions, long_run_average_cost_history, count_state_action_visits


def QL_optimal_policy(pars):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = pars
    # Scales the parameters so that sum of arrival and departure rates is smaller than 1.
    # scale = lam + M*mu*(speed_up(p1, cores) + speed_up(p2, cores))
    scale = lam + mu*np.max([min(M, c)*speed_up(p1, c) + min(M, cores-c)*speed_up(p2, cores-c) for c in range(cores+1)])
    model_pars = [lam/scale, mu/scale, cores, p1, p2, alpha, M]
    # Returns Q-learning optimal policy
    initial_state = [2,2]
    n_iters = 100000
    QL_Q_values, QL_pi_optimal, QL_long_run_average_cost_history, QL_count_state_action_visits = Q_learning(model_pars, initial_state, n_iters)
    
    return QL_Q_values, QL_pi_optimal, QL_long_run_average_cost_history, QL_count_state_action_visits

## SARSA functions

In [6]:
def SARSA(model_pars, initial_state, n_iters):
    lam, mu, cores, p1, p2, alpha, M = model_pars
    
    current_state = initial_state
    next_state = np.zeros(2)
    current_action = np.random.randint(low = 0, high = cores+1, size = 1)[0]/cores
    next_action = 0
    
    Q_values = np.zeros((M+1, M+1, cores+1))
    long_run_average_cost = 0
    state_history = [current_state]
    long_run_average_cost_history = [long_run_average_cost]
    
    for t in range(n_iters):
        # Immediate cost
        immediate_cost = current_state[0] + current_state[1]
        # Long run average cost update
        beta = 1/math.pow((1+t),1)
        long_run_average_cost = (1-beta)*long_run_average_cost + beta*immediate_cost
        long_run_average_cost_history.append(long_run_average_cost)
        # Then, we observe future state
        next_state = sample_next_state(model_pars, current_state, current_action)
        state_history.append(next_state)
        # Given next state, we take next action
        epsilon = 1 / math.pow((1+t), 0.5)
        ber = np.random.binomial(1, epsilon)
        next_action = np.argmin(np.array(Q_values[next_state[0], next_state[1], :]))/cores if ber == 0 else np.random.randint(low = 0, high = cores+1, size = 1)[0]/cores
        # Q update
        alpha = 1/(1 + 0.01*t)
        Q_values[current_state[0], current_state[1], int(cores*current_action)] \
                = (1-alpha)*Q_values[current_state[0], current_state[1], int(cores*current_action)] \
                   + alpha*(immediate_cost - long_run_average_cost + Q_values[next_state[0], next_state[1], int(cores*next_action)])
        # State update
        current_state = next_state
        current_action = next_action
        
    # Evaluating optimal policy as argmin of Q-values    
    optimal_actions = np.zeros((M+1, M+1))
    for n1 in range(M+1):
        for n2 in range(M+1):
            argmin_index = np.argmin(np.array(Q_values[n1, n2, :]))
            optimal_actions[n1, n2] = argmin_index/cores
            
    return Q_values, optimal_actions, long_run_average_cost_history


def SARSA_optimal_policy(pars):
    # Unpacking
    lam, mu, cores, p1, p2, alpha, M = pars
    # Scales the parameters so that sum of arrival and departure rates is smaller than 1.
    # scale = lam + M*mu*(speed_up(p1, cores) + speed_up(p2, cores))
    scale = lam + mu*np.max([min(M, c)*speed_up(p1, c) + min(M, cores-c)*speed_up(p2, cores-c) for c in range(cores+1)])
    model_pars = [lam/scale, mu/scale, cores, p1, p2, alpha, M]
    # Returns Q-learning optimal policy
    initial_state = [2,2]
    n_iters = 100000
    SARSA_Q_values, SARSA_pi_optimal, SARSA_long_run_average_cost_history = SARSA(model_pars, initial_state, n_iters)
    
    return SARSA_Q_values, SARSA_pi_optimal, SARSA_long_run_average_cost_history